In [20]:
import pyspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('pyspark-examples').getOrCreate()


In [58]:
headers=['MARKET','SERIES','SYMBOL','SECURITY','PREV_CL_PR','OPEN_PRICE','HIGH_PRICE','LOW_PRICE','CLOSE_PRICE',
                'NET_TRDVAL',
'NET_TRDQTY','TRADES','HI_52_WK','LO_52_WK']

In [59]:
df = spark.sparkContext.textFile('sharemarket.csv',4)

In [60]:
df=df.map(lambda x:x.split(',')).map(lambda y : y[0:11]+y[12:15])

In [61]:
df = df.toDF(schema=headers)

In [62]:
df.show()

+------+------+----------+--------------------+----------+----------+----------+---------+-----------+----------+----------+------+--------+--------+
|MARKET|SERIES|    SYMBOL|            SECURITY|PREV_CL_PR|OPEN_PRICE|HIGH_PRICE|LOW_PRICE|CLOSE_PRICE|NET_TRDVAL|NET_TRDQTY|TRADES|HI_52_WK|LO_52_WK|
+------+------+----------+--------------------+----------+----------+----------+---------+-----------+----------+----------+------+--------+--------+
|     N|    N1|      IRFC|BOND 8.00% PA TAX...|      1086|      1085|      1085|  1084.54|       1085|3371959.44|      3108|     8|    1194|    1050|
|     N|    N1|      JNPT|BOND 6.82% PA TAX...|      1001|    1007.1|      1015|   1007.1|       1015|     51370|        51|     2| 1529.99|    1000|
|     N|    N1|      NHAI|BOND 8.20% PA TAX...|   1091.38|      1092|      1092|     1087|    1089.98|3861145.49|      3544|    39|    1140|    1045|
|     N|    N1|      NTPC|8.41%S-R-NCD SERI...|   1198.99|      1135|      1135|     1135|       113

In [63]:
df.createOrReplaceTempView("market")

In [64]:
spark.sql('select * from market limit 10').show()

+------+------+---------+--------------------+----------+----------+----------+---------+-----------+----------+----------+------+--------+--------+
|MARKET|SERIES|   SYMBOL|            SECURITY|PREV_CL_PR|OPEN_PRICE|HIGH_PRICE|LOW_PRICE|CLOSE_PRICE|NET_TRDVAL|NET_TRDQTY|TRADES|HI_52_WK|LO_52_WK|
+------+------+---------+--------------------+----------+----------+----------+---------+-----------+----------+----------+------+--------+--------+
|     N|    N1|     IRFC|BOND 8.00% PA TAX...|      1086|      1085|      1085|  1084.54|       1085|3371959.44|      3108|     8|    1194|    1050|
|     N|    N1|     JNPT|BOND 6.82% PA TAX...|      1001|    1007.1|      1015|   1007.1|       1015|     51370|        51|     2| 1529.99|    1000|
|     N|    N1|     NHAI|BOND 8.20% PA TAX...|   1091.38|      1092|      1092|     1087|    1089.98|3861145.49|      3544|    39|    1140|    1045|
|     N|    N1|     NTPC|8.41%S-R-NCD SERI...|   1198.99|      1135|      1135|     1135|       1135|    1

In [65]:
spark.sql('select count(distinct series) from market').show()

+----------------------+
|count(DISTINCT series)|
+----------------------+
|                    53|
+----------------------+



In [66]:
spark.sql('select distinct series from market').show()

+------+
|series|
+------+
|    N3|
|    N2|
|    N5|
|    N4|
|    N6|
|    N1|
|    NA|
|    NC|
|    N7|
|    N8|
|    N9|
|    NB|
|    NL|
|    NK|
|    NJ|
|    NH|
|    NE|
|    ND|
|    NM|
|    NI|
+------+
only showing top 20 rows



In [67]:
spark.sql('select series,sum(open_price),sum(high_price),sum(low_price),sum(close_price) from market group by series').show()

+------+------------------+------------------+------------------+------------------+
|series|   sum(open_price)|   sum(high_price)|    sum(low_price)|  sum(close_price)|
+------+------------------+------------------+------------------+------------------+
|    N3|           2086.24|2088.6000000000004|            2085.9|           2087.38|
|    N2|          10518.07|           10528.4|10491.460000000001|          10510.92|
|    N5|           8720.56|            8850.5|           8690.01|            8831.0|
|    N4|11936.259999999998|11970.689999999999|          11916.05|          11927.66|
|    N6|           26406.1|          26432.09|26300.449999999997|26325.170000000002|
|    N1|            5407.1|            5415.0| 5398.639999999999|           5409.98|
|    NA|           19748.0|20098.340000000004|           19723.0|          20059.57|
|    NC|          14319.21|           14391.7|           14318.5|           14382.9|
|    N7|           3232.41|           3290.55|           3232.41|

In [68]:
spark.sql('select security,series,max(net_trdqty) from market group by security,series').show()

+--------------------+------+---------------+
|            security|series|max(net_trdqty)|
+--------------------+------+---------------+
|  10.25% SRNCD SR IX|    N9|             10|
|7.34 NCD 16FEB23 ...|    N4|            110|
|7.43% TAX FREE TR...|    NH|             96|
|7.51 NCD 16FEB28 ...|    N5|           2063|
|7.64% TAX FREE TR...|    N2|            101|
|7.64% TAX FREETRI...|    N8|           1173|
|  7.74% TAX FREE NCD|    N7|            282|
|   8.00% SEC RED NCD|    N2|          20461|
|8.10 NCD 05MAR22 ...|    N3|           1639|
|8.12/8.32% TAX FR...|    NI|            705|
|8.20 NCD05MAR27 F...|    N2|              3|
|  8.20% TAX FREE NCD|    N4|           1275|
|  8.30% TAX FREE NCD|    N5|            205|
|8.41%S-R-NCD SERI...|    N1|            100|
|8.48%S-R-NCD SERI...|    N2|            100|
|8.49% SEC NON-CUM...|    N7|         143547|
|8.50% TAX FREE TR...|    N5|            100|
|8.52% TAX FREE TR...|    N4|            123|
|  8.66% TAX FREE NCD|    N2|     

In [85]:
#.Display the series whose sum of all prices greater than the net trade value

spark.sql('select series,net_trdval,(open_price+high_price+low_price+close_price) from market where (open_price+high_price+low_price+close_price)>net_trdval').show()

+------+----------+-------------------------------------------------------+
|series|net_trdval|(((open_price + high_price) + low_price) + close_price)|
+------+----------+-------------------------------------------------------+
|    N2|      3666|                                                 4888.0|
|    N4|   2044.02|                                                4088.04|
|    N5|    1912.4|                                                 3824.8|
|    N6|      1341|                                                 5364.0|
|    NB|      1843|                                                 3686.0|
+------+----------+-------------------------------------------------------+



In [71]:
#.Display the series whose sum of all prices greater than the net trade value

spark.sql('select * from market where (open_price+high_price+low_price+close_price)>net_trdval').show()

+------+------+----------+--------------------+----------+----------+----------+---------+-----------+----------+----------+------+--------+--------+
|MARKET|SERIES|    SYMBOL|            SECURITY|PREV_CL_PR|OPEN_PRICE|HIGH_PRICE|LOW_PRICE|CLOSE_PRICE|NET_TRDVAL|NET_TRDQTY|TRADES|HI_52_WK|LO_52_WK|
+------+------+----------+--------------------+----------+----------+----------+---------+-----------+----------+----------+------+--------+--------+
|     N|    N2|     HUDCO|8.20 NCD05MAR27 F...|      1215|      1222|      1222|     1222|       1222|      3666|         3|     2|    1311|    1171|
|     N|    N4|DHANILOANS|DHANI LOANS & SER...|    1020.8|   1022.01|   1022.01|  1022.01|    1022.01|   2044.02|         2|     1| 1098.75|     851|
|     N|    N5|TATACAPHSG|SEC RED NCD 8.20%...|      1000|     912.4|      1000|    912.4|       1000|    1912.4|         2|     2|    1000|   912.4|
|     N|    N6|     HUDCO|BOND 7.03% PA TAX...|      1336|      1341|      1341|     1341|       134

In [72]:
#6.Display the series with highest net trade quantity.(Using pyspark)
spark.sql('select series,net_trdqty from market order by net_trdqty desc limit 1').show()

+------+----------+
|series|net_trdqty|
+------+----------+
|    NH|       989|
+------+----------+



In [78]:
#7.7. Display the highest and lowest open price(Using sql)
spark.sql('select max(int(open_price)),min(int(open_price)) from market').show()

+---------------+---------------+
|max(open_price)|min(open_price)|
+---------------+---------------+
|          11450|             13|
+---------------+---------------+



In [79]:
#8.Query to display the series which have trades more than 80.
spark.sql('select * from market where trades>10').show()

+------+------+----------+--------------------+----------+----------+----------+---------+-----------+----------+----------+------+--------+--------+
|MARKET|SERIES|    SYMBOL|            SECURITY|PREV_CL_PR|OPEN_PRICE|HIGH_PRICE|LOW_PRICE|CLOSE_PRICE|NET_TRDVAL|NET_TRDQTY|TRADES|HI_52_WK|LO_52_WK|
+------+------+----------+--------------------+----------+----------+----------+---------+-----------+----------+----------+------+--------+--------+
|     N|    N1|      NHAI|BOND 8.20% PA TAX...|   1091.38|      1092|      1092|     1087|    1089.98|3861145.49|      3544|    39|    1140|    1045|
|     N|    N2| BRITANNIA|   8.00% SEC RED NCD|     33.07|     33.07|      33.2|     32.8|      33.14| 677372.39|     20461|   107|      36|    29.9|
|     N|    N2|    M&MFIN| 9.00% UNSECURED NCD|      1104|   1101.98|   1101.98|  1080.05|    1089.43|  423116.5|       388|    12|    1229|  1049.5|
|     N|    N2|     PCHFL|SEC RE NCD 8.35% ...|      1000|      1000|   1004.67|     1000|    1001.6

In [86]:
spark.sql('select series from market where trades>80').show()

+------+
|series|
+------+
|    N2|
|    N3|
|    N7|
+------+



In [91]:
#9.Display the difference between the net trade value net trade quantity for each series.(Using sql)
spark.sql('select  series,sum(net_trdval)-sum(net_Trdqty) from market group by series ').show()

+------+-----------------------------------+
|series|(sum(net_trdval) - sum(net_Trdqty))|
+------+-----------------------------------+
|    N3|                         5894945.91|
|    N2|                         3674571.25|
|    N5|                         5581855.67|
|    N4|                 3474029.2199999997|
|    N6|                      1.678578209E7|
|    N1|                         7499601.93|
|    NA|                         3331953.97|
|    NC|                         2505038.79|
|    N7|                         2218463.04|
|    N8|                         3982020.85|
|    N9|                           455370.9|
|    NB|                         2430891.59|
|    NL|                         1581525.31|
|    NK|                         1010121.69|
|    NJ|                         7421503.24|
|    NH|                         1686244.58|
|    NE|                         2538813.87|
|    ND|                 4354764.4399999995|
|    NM|                           583166.7|
|    NI|  